In [46]:
import torch
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm
import copy
from matplotlib import pyplot as plt

## Create single checkup datasets

In [47]:
def create_checkup_datasets():
    
    init_temps = [270.15, 283.15, 296.15, 307.65, 318.65]
    t_amb = [270.15, 283.15, 296.15, 307.65, 318.15]
    
    ground_list = []
    sim_list = []

    for i in range(5):
        file_name = str(10*i) + '.csv'
        ground_list.append(pd.read_csv("nn_data/ground_{}".format(file_name)))
        temp_1 = ground_list[i]['temperature'].tolist()
        temp_1.insert(0, init_temps[i])
        ground_list[i]['t_amb'] = [t_amb[i]] * len(ground_list[i])
        ground_list[i]['lag-1'] = ground_list[i]['temperature'].shift(1)
        ground_list[i]['lag-2'] = ground_list[i]['lag-1'].shift(1)
        ground_list[i]['lag-3'] = ground_list[i]['lag-2'].shift(1)
        ground_list[i]['lag-4'] = ground_list[i]['lag-3'].shift(1)
        ground_list[i]['lag-5'] = ground_list[i]['lag-4'].shift(1)
        ground_list[i]['rolling_5'] = ground_list[i]['temperature'].rolling(window=5, min_periods=1).mean()
        ground_list[i]['rolling_10'] = ground_list[i]['temperature'].rolling(window=20, min_periods=1).mean()
        ground_list[i]['rolling_25'] = ground_list[i]['temperature'].rolling(window=5, min_periods=1).mean()
        ground_list[i]['t_amb'] = [t_amb[i]] * len(ground_list[i])

        sim_list.append(pd.read_csv("nn_data/dataset_{}".format(file_name)))
        ground_list[i]['heat'] = sim_list[i]['heat'].tolist()[1:]
        ground_list[i]['voltage_sim'] = sim_list[i]['voltage'].tolist()[1:]
        
        ground_list[i].to_csv('nn_data/train_{}'.format(file_name), index=False)
    
    return ground_list

In [48]:
datasets = create_checkup_datasets()

In [49]:
ground = pd.concat(datasets, ignore_index=True)
ground.to_csv('nn_data/full_train.csv', index=False)

## Create PV1 and PV2 datasets

In [50]:
def create_pv_datasets():
    init_temps = [295.15, 295.15]
    t_amb = [295.15, 295.15]
    
    ground_list = []
    sim_list = []

    for i in range(2):
        file_name = 'pv' + str(i+1) + '.csv'
        ground_list.append(pd.read_csv("nn_data/ground_{}".format(file_name)))
        temp_1 = ground_list[i]['temperature'].tolist()
        temp_1.insert(0, init_temps[i])
        ground_list[i]['t_amb'] = [t_amb[i]] * len(ground_list[i])
        ground_list[i]['lag-1'] = ground_list[i]['temperature'].shift(1)
        ground_list[i]['lag-2'] = ground_list[i]['lag-1'].shift(1)
        ground_list[i]['lag-3'] = ground_list[i]['lag-2'].shift(1)
        ground_list[i]['lag-4'] = ground_list[i]['lag-3'].shift(1)
        ground_list[i]['lag-5'] = ground_list[i]['lag-4'].shift(1)
        ground_list[i]['rolling_5'] = ground_list[i]['temperature'].rolling(window=5, min_periods=1).mean()
        ground_list[i]['rolling_10'] = ground_list[i]['temperature'].rolling(window=20, min_periods=1).mean()
        ground_list[i]['rolling_25'] = ground_list[i]['temperature'].rolling(window=5, min_periods=1).mean()
        ground_list[i]['t_amb'] = [t_amb[i]] * len(ground_list[i])

        sim_list.append(pd.read_csv("nn_data/dataset_{}".format(file_name)))
        ground_list[i]['heat'] = sim_list[i]['heat'].tolist()[1:]
        ground_list[i]['voltage_sim'] = sim_list[i]['voltage'].tolist()[1:]
        
        ground_list[i].to_csv('nn_data/train_{}'.format(file_name), index=False)
    
    return ground_list

In [51]:
pv_datasets = create_pv_datasets()

In [52]:
pv_datasets

[        current  voltage  temperature      power      time   t_amb     lag-1  \
 0       10.0085   3.5802     294.6195  35.832684       0.0  295.15       NaN   
 1       10.0056   3.5905     294.5774  35.925229       1.0  295.15  294.6195   
 2       10.0056   3.5921     294.5694  35.940675       2.0  295.15  294.5774   
 3       10.0051   3.5936     294.5845  35.954372       3.0  295.15  294.5694   
 4       10.0089   3.5946     294.6684  35.978527       4.0  295.15  294.5845   
 ...         ...      ...          ...        ...       ...     ...       ...   
 100582  -0.2434   3.7304     295.1057  -0.908100  100582.0  295.15  295.0694   
 100583  -0.2434   3.7305     295.1095  -0.908131  100583.0  295.15  295.1057   
 100584  -0.2231   3.7303     295.0694  -0.832411  100584.0  295.15  295.1095   
 100585  -0.2322   3.7305     295.1321  -0.866088  100585.0  295.15  295.0694   
 100586  -0.2502   3.7302     295.1248  -0.933277  100586.0  295.15  295.1321   
 
            lag-2     lag-